# User routes on the site
## Description
**Clickstream** is a sequence of user actions on a website. It allows you to understand how users interact with the site. In this task, you need to find the most frequent custom routes.

## Input data
Input data is а table with clickstream data in file `hdfs:/data/clickstream.csv`.

### Table structure
* `user_id (int)` - Unique user identifier.
* `session_id (int)` - Unique identifier for the user session. The user's session lasts until the identifier changes.
* `event_type (string)` - Event type from the list:
    * **page** - visit to the page
    * **event** - any action on the page
    * <b>&lt;custom&gt;</b> - string with any other type
* `event_type (string)` - Page on the site.
* `timestamp (int)` - Unix-timestamp of action.

### Browser errors
Errors can sometimes occur in the user's browser - after such an error appears, we can no longer trust the data of this session and all the following lines after the error or at the same time with it are considered corrupted and **should not be counted** in statistics.

When an error occurs on the page, a random string containing the word **error** will be written to the `event_type` field.

### Sample of user session
<pre>
+-------+----------+------------+----------+----------+
|user_id|session_id|  event_type|event_page| timestamp|
+-------+----------+------------+----------+----------+
|    562|       507|        page|      main|1620494781|
|    562|       507|       event|      main|1620494788|
|    562|       507|       event|      main|1620494798|
|    562|       507|        page|    family|1620494820|
|    562|       507|       event|    family|1620494828|
|    562|       507|        page|      main|1620494848|
|    562|       507|wNaxLlerrorU|      main|1620494865|
|    562|       507|       event|      main|1620494873|
|    562|       507|        page|      news|1620494875|
|    562|       507|        page|   tariffs|1620494876|
|    562|       507|       event|   tariffs|1620494884|
|    562|       514|        page|      main|1620728918|
|    562|       514|       event|      main|1620729174|
|    562|       514|        page|   archive|1620729674|
|    562|       514|        page|     bonus|1620729797|
|    562|       514|        page|   tariffs|1620731090|
|    562|       514|       event|   tariffs|1620731187|
+-------+----------+------------+----------+----------+
</pre>

#### Correct user routes for a given user:
* **Session 507**: main-family-main
* **Session 514**: main-archive-bonus-tariffs

Route elements are ordered by the time they appear in the clickstream, from earliest to latest.

The route must be accounted for completely before the end of the session or an error in the session.

## Task
You need to use the Spark SQL, Spark RDD and Spark DF interfaces to create a solution file, the lines of which contain **the 30 most frequent user routes** on the site.

Each line of the file should contain the `route` and `count` values **separated by tabs**, where:
* `route` - route on the site, consisting of pages separated by "-".
* `count` - the number of user sessions in which this route was.

The lines must be **ordered in descending order** of the `count` field.

## Criteria
You can get maximum of 3.5 points (final grade) for this assignment, depedning on the number of interface you manage to leverage. The criteria are as follows:

* 0.5 points – Spark SQL solution with 1 query
* 0.5 points – Spark SQL solution with <=2 queries
* 0.5 points – Spark RDD solution
* 0.5 points – Spark DF solution
* 0.5 points – your solution algorithm is relatively optimized, i.e.: no O^2 or O^3 complexities; appropriate object usage; no data leaks etc. This is evaluated by staff.
* 1 point – 1 on 1 screening session. During this session staff member can ask you questions regarding your solution logic, framework usage, questionable parts of your code etc. If your code is clean enough, the staff member can just ask you to solve a theoretical problem connected to Spark.


In [ ]:
# Here we need to archive the modules folder with all its contents to pass the functions to the workers,
# since the workers do not see the functions when using UDF and map (since the Python files are running locally)
# But I added the archive to the project

# import shutil
# shutil.make_archive('modules', 'zip', '/home/jovyan/work/modules')

In [1]:
from modules import spark_init

In [2]:
APPNAME = 'sga_pavlov'
SOURCE_PATH = 'file:///home/jovyan/work/clickstream.csv'
OUTPUT_PATH = '/home/jovyan/work/result.json'
VIEW_NAME = 'clickstream'

In [3]:
spark, sparkContext = spark_init.create_spark(APPNAME)

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
2024-11-05 18:18:21,802 WARN yarn.Client: Neither spark.yarn.jars nor spark.yarn.archive is set, falling back to uploading libraries under SPARK_HOME.


In [4]:
import json
from modules import loads
from modules import preprocessing

### Spark DataFrame Solution

In [5]:
clickstream_sdf = loads.load_to_sdf(spark, SOURCE_PATH)

In [6]:
# I checked that there are no leading zeros in all three columns and this will not distort our data. But the operations will be faster
clickstream_sdf = preprocessing.get_true_dtypes(clickstream_sdf)
clickstream_sdf.cache()
clickstream_sdf.count()

1000000

In [7]:
clickstream_sdf.show(10, False)

+-------+----------+------------+----------+----------+
|user_id|session_id|event_type  |event_page|timestamp |
+-------+----------+------------+----------+----------+
|562    |507       |page        |main      |1695584127|
|562    |507       |event       |main      |1695584134|
|562    |507       |event       |main      |1695584144|
|562    |507       |event       |main      |1695584147|
|562    |507       |wNaxLlerrorU|main      |1695584154|
|562    |507       |event       |main      |1695584154|
|562    |507       |event       |main      |1695584154|
|562    |507       |event       |main      |1695584160|
|562    |507       |page        |rabota    |1695584166|
|562    |507       |event       |rabota    |1695584174|
+-------+----------+------------+----------+----------+
only showing top 10 rows



In [8]:
clickstream_sdf = preprocessing.exclude_error_events(clickstream_sdf)
clickstream_sdf = preprocessing.exclude_same_pages(clickstream_sdf)

In [9]:
routes_count_sdf = preprocessing.collect_route(clickstream_sdf)

In [10]:
# Select the top 10 routes
top_10_routes_sdf = preprocessing.get_top_n_routes_df(routes_count_sdf, 10)
top_10_routes_sdf.show(10, False)

+-------------------+-----+
|route              |count|
+-------------------+-----+
|main               |8184 |
|main-archive       |1113 |
|main-rabota        |1047 |
|main-internet      |897  |
|main-bonus         |870  |
|main-news          |769  |
|main-tariffs       |677  |
|main-online        |587  |
|main-vklad         |518  |
|main-rabota-archive|170  |
+-------------------+-----+



In [11]:
result = {row['route']: row['count'] for row in top_10_routes_sdf.collect()}
result

{'main': 8184,
 'main-archive': 1113,
 'main-rabota': 1047,
 'main-internet': 897,
 'main-bonus': 870,
 'main-news': 769,
 'main-tariffs': 677,
 'main-online': 587,
 'main-vklad': 518,
 'main-rabota-archive': 170}

### Spark SQL Solution

In [12]:
loads.load_to_tempview(spark, SOURCE_PATH, VIEW_NAME)

In [13]:
sql_query = preprocessing.get_ready_sql_query()

top_10_routes_sdf = spark.sql(sql_query)
top_10_routes_sdf.show(10, False)

+-------------------+-----+
|route              |count|
+-------------------+-----+
|main               |8184 |
|main-archive       |1113 |
|main-rabota        |1047 |
|main-internet      |897  |
|main-bonus         |870  |
|main-news          |769  |
|main-tariffs       |677  |
|main-online        |587  |
|main-vklad         |518  |
|main-rabota-archive|170  |
+-------------------+-----+



### Spark RDD Solution

In [14]:
# Read the file into an RDD
records = loads.load_to_rdd(sparkContext, SOURCE_PATH)

In [15]:
records.take(10)

[('562', '507', 'page', 'main', '1695584127'),
 ('562', '507', 'event', 'main', '1695584134'),
 ('562', '507', 'event', 'main', '1695584144'),
 ('562', '507', 'event', 'main', '1695584147'),
 ('562', '507', 'wNaxLlerrorU', 'main', '1695584154'),
 ('562', '507', 'event', 'main', '1695584154'),
 ('562', '507', 'event', 'main', '1695584154'),
 ('562', '507', 'event', 'main', '1695584160'),
 ('562', '507', 'page', 'rabota', '1695584166'),
 ('562', '507', 'event', 'rabota', '1695584174')]

In [16]:
routes = preprocessing.get_routes(records)
top_routes = preprocessing.get_top_n_routes_rdd(routes, 10)

In [17]:
# Print the top 10 routes with their counts
for route, count in top_routes:
    print(f"{route}\t{count}")

main	8185
main-archive	1113
main-rabota	1047
main-internet	897
main-bonus	870
main-news	769
main-tariffs	677
main-online	587
main-vklad	518
main-rabota-archive	170


### Output to grader

In [18]:
with open(OUTPUT_PATH, "w") as json_file:
    json.dump(result, json_file, indent=4)

In [16]:
# !curl -F file=@result.json 51.250.123.136:80/MDS-LSML1/Cane4er/w6/2

0.9999999999999999
Correct main answer!
Correct main-archive answer!
Correct main-rabota answer!
Correct main-internet answer!
Correct main-bonus answer!
Correct main-news answer!
Correct main-tariffs answer!
Correct main-online answer!
Correct main-vklad answer!
Correct main-rabota-archive answer!
